## Network Validation Checks

#### Import Libraries 

In [ ]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# from nxviz import CircosPlot

#### Load Network data from Shapefiles

In [ ]:
line_shp_load = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master.shp')).to_crs(epsg=2274)
node_shp_load = gpd.read_file(os.path.join('..', 'data', 'Chatt_Master_Node.shp')).to_crs(epsg=2274)

### Build Network Graph

#### Turn Two-Way Links into 2 One-Way Links for Directional Flow.
#### Create Function: *directional_links*
Keep AB Links and produce BA Links, carrying over link attributes based on the direction of flow.

In [ ]:
def directional_links(atts, line_shp):
    dirs = {'keep':'AB', 'invert':'BA'}
    dfs = {}
    
    for flow,d in dirs.items():
        links = line_shp[line_shp['{}_LANES'.format(d)] > 0].copy()
        rename_dict = {'{}{}'.format(d, a):a for a in atts}
        links = links.rename(columns=rename_dict)
        
        if flow == 'invert':
            links['A'] = links['TO_ID']
            links['B'] = links['FROM_ID']
            links = links.drop(columns=['FROM_ID', 'TO_ID']).rename(columns={'A': 'FROM_ID', 'B': 'TO_ID'})
        
        dfs['{}_links'.format(d)] = links
    
    links = pd.concat(dfs.values(), ignore_index=True)
    
    drop_dict = ['{}{}'.format(d, a) for a in atts for d in dirs.values()]
    links = links.drop(columns=drop_dict)
    
    return links

In [ ]:
line_shp[line_shp['ID']==560]

#### Generate Directional links with Attributes
####  Call Function: *directional_links*
Use list of Directional Attributes to carry over, e.g. 'AB_LANES' and 'BA_LANES', to '_LANES' 

In [ ]:
atts = [
    '_LINKDIR',
    '_LINKDTR',
    '_LANES',
    '_PARKING',
    '_TRIMS',
    '_AADT',
    '_CAR_ADT',
    '_SUT_ADT',
    '_MUT_ADT',
    '_BASEVOL',
    '_AFFTIME',
    '_AFFSPD',
    '_UCDELAY',
    '_DLYCAP',
    '_AMCAP',
    '_PMCAP',
    '_BPRA',
    '_BPRB'
]

line_shp = directional_links(atts, line_shp_load)

####  Build the Directional Graph: *g*
#### Call Method: *nx.from_pandas_edgelist*

In [ ]:
g = nx.from_pandas_edgelist(
    line_shp,
    source='FROM_ID',
    target='TO_ID',
    edge_attr=True,
    create_using=nx.DiGraph()
)

assert nx.is_directed(g)

### Link to Link validation Functions

#### Create Function: *link_matcher_diff*: 
For each node in Network, match all combinations of In/ Out Links and return difference (Substraction)

In [ ]:
def link_matcher_diff(*atts, graph, edge_id='ID'):
    node_feed = []

    for node in node_shp_load['ID']:
        if node not in g:
            print('Node not in network: {}'.format(node))
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]                
                node_feed.append([node, *[d[edge_id] for d in data], *[o_data[a] - i_data[a] for a in atts]])

    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node', *[ d+'_'+edge_id for d in dirs], *['diff_'+a for a in atts]]

    return pd.DataFrame(node_feed, columns=cols)

#### Create Function: *link_matcher_atts*: 
For each node in Network, match all combinations of In/ Out Links and return both In and Out values

In [ ]:
def link_matcher_atts(*atts, graph, edge_id='ID'):
    node_feed = []

    for node in node_shp_load['ID']:
        if node not in g:
            print('Node not in network: {}'.format(node))
            continue

        for i_o, i_d, i_data  in g.in_edges(node, data=True):
            for o_o, o_d, o_data in g.out_edges(node, data=True):
                if i_o == o_d and i_d == o_o:
                    # This is just a U-Turn on the "same" link
                    continue
                    
                data = [i_data, o_data]                
                node_feed.append([node, *[d[edge_id] for d in data], *[d[a] for a in atts for d in data]])

    # Generate Column names
    dirs = ['in', 'out']
    cols = ['node', *[ d+'_'+edge_id for d in dirs], *[ d+'_'+a for a in atts for d in dirs]]

    return pd.DataFrame(node_feed, columns=cols)

#### Create Function: *attribute_filter_diff*: 
Apply Attribute to filter by difference value 

In [ ]:
def attribute_filter_diff(network, att_name, att_filter_val, get_node_max):
    fld_diff = 'diff_{}'.format(att_name)
    abs_fld_diff = 'abs_diff_{}'.format(att_name)

    network = network[abs(network[fld_diff])>=att_filter_val].reset_index(drop=True)
    network[abs_fld_diff] = abs(network[fld_diff])

    if get_node_max:
        network['sorted'] = network.sort_values([abs_fld_diff]).groupby('node').cumcount()+1
        network = network[network['sorted']==1][['node', 'in_ID', 'out_ID', fld_diff]]

    return network.sort_values(by=fld_diff, ascending=False)

#### Create Function: *attribute_filter_atts*: 
Apply Attribute to filter by difference value 

In [ ]:
def attribute_filter_atts(att_names, network, att_filter_name, att_filter_val, get_node_max):
    in_fld = 'in_{}'.format(att_filter_name)
    out_fld = 'out_{}'.format(att_filter_name)
    fld_diff = 'diff_{}'.format(att_filter_name)
#     abs_fld_diff = 'abs_diff_{}'.format(att_name)
    
    if isinstance(att_filter_val, (int, float)):
        network = network[abs(network[out_fld] - network[in_fld]) >= att_filter_val]
    elif isinstance(att_filter_val, str):
        network = network[(network[in_fld] == att_filter_val) | (network[out_fld] == att_filter_val)]
        
    return network#.groupby([in_fld, out_fld]).count().reset_index()

#### Create Function: *export_geo*: 
Export Link To/From values, from Node Maximum

In [ ]:
# In/Out Long Table
def export_geo(network, att_name, att_name_add=None):
    fld_diff = 'diff_{}'.format(att_name)
    
    in_df = network[['node', 'in_ID']].copy()
    in_df.columns = ['node', 'line_ID']

    network = network[['node', 'out_ID', fld_diff]].copy()
    network.columns = ['node', 'line_ID', fld_diff]

    network = network.append(in_df)

    # geometries
    geo_cols = ['ID', 'geometry']
    if att_name_add: geo_cols.append(att_name_add)
    network = pd.merge(line_shp_load[geo_cols], network, left_on='ID', right_on='line_ID')
    
    shp_export = os.path.join('..', 'data', 'Network_attribute_change_{}.shp'.format(att_name))
    network.to_file(shp_export)
    return network.sort_values(by=fld_diff, ascending=False), shp_export

In [ ]:
# In/Out Long Table
def export_geo_compare(network, att_name, att_name_add=None):
    fld_diff = 'diff_{}'.format(att_name)
    
    network = network.rename(columns={'out_ID':'ID'})

    # geometries
    geo_cols = ['ID', 'geometry']
#     if att_name_add: geo_cols.append(att_name_add)
    network = pd.merge(line_shp_load[geo_cols], network, on='ID')
    
    shp_export = os.path.join('..', 'data', 'Network_attribute_change_{}.shp'.format('_'.join([n for n in att_name])))
    network.to_file(shp_export)
#     return network#.sort_values(by=fld_diff, ascending=False), shp_export
    return network, shp_export

#### Create Function: *link_difference*: 
Call functions to calculate Link-to-Link attribute difference

In [ ]:
def link_difference(attribute_name, attribute_filter_value, get_node_max, additional_attribute_name):
    network = link_matcher_diff(attribute_name, graph=g)
    network = attribute_filter_diff(network, attribute_name, attribute_filter_value, get_node_max)
    csv_export = os.path.join('..', 'data', 'Network_attribute_change_table_{}.csv'.format(attribute_name))
    network.to_csv(csv_export, index=False)
    
    # Print
    print_txt = 'Total {} network links with change in {} of {} or greater: {:,}'
    print(print_txt.format('SUMMARIZED' if get_node_max else '\b',
                           attribute_name,
                           attribute_filter_value,
                           len(network)
                           ))
    print('Table exported to {}'.format(csv_export))
    if get_node_max:
        network, shp_export = export_geo(network, attribute_name, additional_attribute_name)
        print('Shapefile exported to {}'.format(shp_export))

    return network

#### Create Function: *link_compare*: 
Call functions to calculate Link-to-Link attribute comparison

In [ ]:
def link_compare(attributes_names, attribute_filter_name, attribute_filter_value, get_node_max):
    network = link_matcher_atts(*attributes_names, graph=g)
    network = attribute_filter_atts(
                                    attributes_names,
                                    network,
                                    attribute_filter_name,
                                    attribute_filter_value,
                                    get_node_max
                                   )
    csv_export = os.path.join('..', 'data', 'Network_attribute_change_table_{}.csv'.format('_'.join([n for n in attributes_names])))
    network.to_csv(csv_export, index=False)
    
    # Print
    print_txt = 'Total {} network links with change in {} of {} or greater: {:,}'
    print(print_txt.format('SUMMARIZED' if get_node_max else '\b',
                           attribute_filter_name,
                           attribute_filter_value,
                           len(network)
                           ))
    print('Table exported to {}'.format(csv_export))
    if get_node_max:
        network, shp_export = export_geo_compare(network, attributes_names, attribute_filter_name)
        print('Shapefile exported to {}'.format(shp_export))

    return network

## VALIDATION TESTS

### Network Link-to-Link attribute difference validation : SPEED LIMIT

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_name = 'SPD_LMT'
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filter_value = 10
# Summarize by Node Maximum value and Export Geo:
get_node_max = True
additional_attribute_name = 'RTE_NME'


### RUN FUNCTIONS
network = link_difference(attribute_name, attribute_filter_value, get_node_max, additional_attribute_name)
network.head()

### Network Link-to-Link attribute difference validation : NUMBER OF LANES

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attribute_name = '_LANES'
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filter_value = 2
# Summarize by Node Maximum value and Export Geo:
get_node_max = True
additional_attribute_name = 'FUNCCLASS'


### RUN FUNCTIONS
network = link_difference(attribute_name, attribute_filter_value, get_node_max, additional_attribute_name)
network.head()

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and FUNCTIONAL CLASS

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attributes_names = ['SPD_LMT', 'FUNCCLASS']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filter_name = 'SPD_LMT'
attribute_filter_value = 10
# Summarize by Node Maximum value and Export Geo:
get_node_max = True


### RUN FUNCTIONS
network = link_compare(attributes_names, attribute_filter_name, attribute_filter_value, get_node_max)
network.head()

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Speed Limit

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attributes_names = ['SPD_LMT', 'RTE_NME']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filter_name = 'SPD_LMT'
attribute_filter_value = 10
# Summarize by Node Maximum value and Export Geo:
get_node_max = True


### RUN FUNCTIONS
network = link_compare(attributes_names, attribute_filter_name, attribute_filter_value, get_node_max)
network.head()

### Network Link-to-Link attribute side-by-side validation: SPEED LIMIT and ROUTE NAME
Filter by Route Name

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attributes_names = ['SPD_LMT', 'RTE_NME']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filter_name = 'RTE_NME'
attribute_filter_value = 'CENTROID CONNECTOR'
# Summarize by Node Maximum value and Export Geo:
get_node_max = False


### RUN FUNCTIONS
network = link_compare(attributes_names, attribute_filter_name, attribute_filter_value, get_node_max)
network.head()

### Network Link-to-Link attribute side-by-side validation: NUMBER OF LANES and FUNCTIONAL CLASS
Filter by Route Name

In [ ]:
### SPECIFY PARAMETERS
# Attribute to validate:
attributes_names = ['_LANES', 'FUNCCLASS']
# Filter by attribute value (Equal to or Greater than) and Export long table:
attribute_filter_name = '_LANES'
attribute_filter_value = 2
# Summarize by Node Maximum value and Export Geo:
get_node_max = True


### RUN FUNCTIONS
network = link_compare(attributes_names, attribute_filter_name, attribute_filter_value, get_node_max)
network.head()

### Put attributes on the network

In [ ]:
node_shp['X'] = node_shp.geometry.x
node_shp['Y'] = node_shp.geometry.y
nx.set_node_attributes(g, pd.Series(node_shp.X, index=node_shp.ID).to_dict(), 'X')
nx.set_node_attributes(g, pd.Series(node_shp.Y, index=node_shp.ID).to_dict(), 'Y')

### Visualizations (Slow- Due to Network Size)

In [ ]:
# plt.subplot(121)

#nx.draw(g, with_labels=True, font_weight='bold')
#plt.subplot(122)

#nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(g, nx.spring_layout(g), node_size=10, node_color='blue')
plt.show()
